# Clasificación de imágenes con PyTorch

Versión de: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

## Cargando los datos

PyTorch cuenta con las librerías [TorchText](https://pytorch.org/text/stable/index.html), [TorchVision](https://pytorch.org/vision/stable/index.html), y [TorchAudio](https://pytorch.org/audio/stable/index.html) para cargar y manipular datos. En este caso, usaremos TorchVision para cargar el dataset [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist).

FAshionMNIST es un dataset de ropa que contiene 60,000 imágenes de entrenamiento y 10,000 imágenes de prueba. Cada imagen es de 28x28 píxeles y pertenece a una de las [10 clases de ropa](https://github.com/zalandoresearch/fashion-mnist?tab=readme-ov-file#labels). Funciona como un reemplazo directo para el dataset MNIST, que es más comúnmente usado para probar algoritmos de aprendizaje automático.

Estos conjuntos de datos son subclases de `torch.utils.data.Dataset`. Cada [`Dataset` de TorchVision](https://pytorch.org/vision/stable/datasets.html) incluye dos argumentos: `transform` y `target_transform` para modificar las muestras y las etiquetas respectivamente.

Las imágenes se descargan en formato PIL, y las transformaciones se aplican en el momento de cargarlas. En este caso, se convierten a tensores de PyTorch con la transformación `ToTensor`.

In [117]:
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST( # Construye un objeto FashionMNIST (subclase de torch.utils.data.Dataset)
    root="data", # directorio donde se almacenan los datos
    train=True, # carga el conjunto de entrenamiento
    download=True,  # descarga el conjunto de datos si no está en el directorio de datos
    transform=ToTensor(), # ToTensor convierte la imagen en un tensor de PyTorch
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [118]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [119]:
training_data.data.shape

torch.Size([60000, 28, 28])

In [120]:
training_data.targets

tensor([9, 0, 0,  ..., 3, 0, 5])

El `Dataset` se pasa como argumento a `DataLoader`, una clase que envuelve el dataset haciéndolo iterable, y que añade los procesos de *batching* (creación de lotes), *sampling* (muestreo), *shuffling* (mezcla) y carga automática de datos en múltiples procesos. Aquí definimos un tamaño de *batch* de 64, es decir, cada elemento en el iterable del dataloader devolverá un *batch* de 64 características y etiquetas.

Un *batch* es un lote de muestras que se procesan en cada iteración del entrenamiento. En cada iteración, el modelo recibe un *batch* de muestras, calcula las predicciones y la función de pérdida (***loss function***), y actualiza los pesos del modelo haciendo ***backpropagation*** para minimizar la pérdida. Típicamente, se usan tamños de batch de 64, 128, 256 o 512 (potencias de 2 para que el tamaño del batch se ajuste a la memoria de la GPU y se procese de manera eficiente).

In [121]:
import torch
from torch.utils.data import DataLoader

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N(number os samples), C(color_channels), H(height), W(width)]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N(number os samples), C(color_channels), H(height), W(width)]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creando modelos

El paquete `torch.nn` contiene las herramientas necesarias para definir redes neuronales en PyTorch. En particular, cualquier red neuronal que queramos definir debe ser una clase que herede de [`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).

Toda clase que herede de `torch.nn.Module` debe implementar dos métodos:
- `__init__`: Constructor de la clase. Aquí se definen las capas de la red.
- `forward`: Método que define cómo se calcula la salida de la red a partir de la entrada.

In [122]:
import torch.nn as nn

class NeuralNetwork(nn.Module): # Clase que hereda de nn.Module y define la arquitectura de la red
    def __init__(self): # Constructor de la clase
        super().__init__() # Llama al constructor de la clase padre
        self.flatten = nn.Flatten() # Capa de aplanamiento de la imagen (28x28 -> 784)
        self.linear_relu_stack = nn.Sequential( # Secuencia de capas lineales y funciones de activación ReLU
            nn.Linear(28*28, 512), # Capa de entrada con 784 entradas y 512 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa de entrada
            nn.Linear(512, 512), # Capa oculta totalmente conectada con 512 entradas y 512 salidas
            nn.ReLU(), # Función de activación ReLU después de la capa oculta
            nn.Linear(512, 10) # Capa de salida con 512 entradas y 10 salidas
        )

    def forward(self, x): # Método que define el flujo de datos a través de la red
        x = self.flatten(x) # Aplana la imagen
        logits = self.linear_relu_stack(x) # Pasa los datos a través de la secuencia de capas
        return logits # Devuelve los logits (salida sin activación)
    
model = NeuralNetwork() # Instancia del modelo

La capa de entrada tendrá necesariamente 784 neuronas, una por cada pixel de la imagen de entrada (recordemos que las imágenes de MNIST son de 28x28 píxeles). Para pasar los tensores de imágenes de 28x28 píxeles a un tensor de 784 píxeles, usamos la capa `Flatten`, que solo cambia la forma de los datos.

La capa de salida tendrá 10 neuronas, una por cada posible dígito al que puede corresponder la imagen de entrada.

Definimos en medio dos **capas ocultas (*hidden layers*)** de 512 neuronas cada una. La función de activación de las capas ocultas (para cada una de sus neuronas) es la función **ReLU** (la más común en redes neuronales).

`Sequential` es un contenedor que apila módulos en el orden en que se pasan a la clase. Cada módulo se aplica a la salida del módulo anterior. En este caso, `Sequential` define la secuencia de capas de la red neuronal.

`Linear` define una capa de red neuronal completamente conectada (también conocida como capa densa). Cada neurona de una capa está conectada a todas las neuronas de la capa anterior. La capa `Linear` requiere dos argumentos: el número de neuronas de entrada y el número de neuronas de salida.


En cada ejecución de `forward`, primero se pasa la entrada a través de la capa `Flatten` para convertir la imagen de 28x28 píxeles en un tensor de 784 píxeles. Luego, la entrada se pasa a través de las capas ocultas, y se aplica la función de activación `ReLU` después de cada capa oculta. Finalmente, la salida de la última capa oculta se pasa a través de la capa de salida, que devuelve un tensor de 10 *logits*. Los *logits* son valores que no han sido normalizados y que se utilizan para calcular las probabilidades de cada clase.

Para acelerar el entrenamiento, PyTorch puede aprovechar la GPU si está disponible. Para ello, se debe mover el modelo y los datos a la GPU con el método `to`.

In [123]:
device = (
    "cuda" if torch.cuda.is_available() 
    else "mps" if torch.backends.mps.is_available()
    else "cpu" 
)
print(f"Using {device} device")

model = model.to(device) # Mueve el modelo a la GPU si está disponible
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimización y entrenamiento

Para optimizar los parámetros del modelo, necesitamos una [**función de pérdida**](https://pytorch.org/docs/stable/nn.html#loss-functions) y un [optimizador](https://pytorch.org/docs/stable/optim.html).

Definimos la función de pérdida `nn.CrossEntropyLoss`, que se utiliza comúnmente en problemas de clasificación. Esta función calcula la pérdida de entropía cruzada entre las predicciones y las etiquetas reales. La entropía cruzada es una medida de la diferencia entre dos distribuciones de probabilidad.

Definimos el optimizador `optim.SGD` (descenso de gradiente estocástico) con una tasa de aprendizaje (***learning rate***) de 0.1. El optimizador ajusta los pesos del modelo en función de la pérdida calculada.

In [124]:
loss_fn = nn.CrossEntropyLoss() # Función de pérdida
optimizer = torch.optim.SGD( # Optimizador de descenso de gradiente estocástico
    model.parameters(), # Parámetros del modelo a optimizar
    lr=1e-3 # Tasa de aprendizaje
    )

Definimos la función `train` que realiza un paso de entrenamiento (una iteración) y la función `test` que evalúa el modelo en el conjunto de prueba.

En cada iteración del bucle de entrenamiento, el modelo hace predicciones sobre el conjunto de entrenamiento (alimentado en lotes), y retropropaga el error de predicción para ajustar los parámetros del modelo.



In [125]:
def train(dataloader, model, loss_fn, optimizer):
    
    size = len(dataloader.dataset) # Número de muestras en el conjunto de datos
    
    model.train() # Pone el modelo en modo de entrenamiento
    for batch_num, (X, y) in enumerate(dataloader): # Itera sobre los lotes de datos, para cada uno:
        X, y = X.to(device), y.to(device) # Mueve el array de datos y las etiquetas al dispositivo

        pred = model(X) # Genera predicciones
        loss = loss_fn(pred, y) # Calcula la pérdida para ese lote

        # Backpropagation
        loss.backward() # Calcula el gradiente de la función de pérdida
        optimizer.step() # Actualiza los parámetros
        optimizer.zero_grad() # Resetea los gradientes

        if batch_num % 100 == 0: # Cada 100 lotes imprime el progreso
            loss, current = loss.item(), (batch_num + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Definimos también una función que evalúa el modelo en el conjunto de prueba.


In [126]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # Pone el modelo en modo de evaluación
    test_loss, correct = 0, 0
    with torch.no_grad(): # Desactiva el cálculo de gradientes para el siguiente bloque de código
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item() # Acumula la pérdida
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # Acumula el número de aciertos [1]
    test_loss /= num_batches # Calcula la pérdida promedio por lote
    correct /= size # Calcula la exactitud (número de aciertos / número total de muestras)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

> [1]: `pred` es un tensor de forma `(batch_size, 10)` con los *logits* de cada clase. Con `pred.argmax(1)`, encontramos el índice con el valor más alto en cada fila (eje 1: columnas), o lo que es lo mismo: la clase con la probabilidad más alta para cada imagen. Comparamos las predicciones con las etiquetas reales (`(pred.argmax(1) == y)`) devolviendo un array de booleanos y lo sumamos con `sum()` para obtener el número de predicciones correctas. Para sumarlos, antes convertimos los booleanos a floats (no se usa 'int' por compatibilidad con funciones de PyTorch). El resultado es un tensor con un solo valor que contiene el número total de predicciones correctas en el *batch*; usamos `.item()` para obtener el valor numérico contenido en ese tensor.

El proceso de entrenamiento se realiza a través de varias iteraciones (*epochs*). Una ***epoch*** es una pasada completa a través de todo el conjunto de datos de entrenamiento. Cada *epoch* se divide en lotes, y el modelo se entrena en cada lote. Después de cada *epoch*, evaluamos el modelo en el conjunto de test.

In [127]:
epochs = 10 # Número de epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.289433  [   64/60000]
loss: 2.283456  [ 6464/60000]
loss: 2.275591  [12864/60000]
loss: 2.278797  [19264/60000]
loss: 2.253232  [25664/60000]
loss: 2.219962  [32064/60000]
loss: 2.234259  [38464/60000]
loss: 2.194916  [44864/60000]
loss: 2.196166  [51264/60000]
loss: 2.172099  [57664/60000]
Test Error: 
 Accuracy: 36.0%, Avg loss: 2.163718 

Epoch 2
-------------------------------
loss: 2.166211  [   64/60000]
loss: 2.159473  [ 6464/60000]
loss: 2.115984  [12864/60000]
loss: 2.137422  [19264/60000]
loss: 2.088166  [25664/60000]
loss: 2.023160  [32064/60000]
loss: 2.057282  [38464/60000]
loss: 1.978723  [44864/60000]
loss: 1.981582  [51264/60000]
loss: 1.918857  [57664/60000]
Test Error: 
 Accuracy: 57.5%, Avg loss: 1.914655 

Epoch 3
-------------------------------
loss: 1.941010  [   64/60000]
loss: 1.909618  [ 6464/60000]
loss: 1.809062  [12864/60000]
loss: 1.851439  [19264/60000]
loss: 1.742894  [25664/60000]
loss: 1.685593  [32064/600

## Guardando y cargando el modelo entrenado

El entrenamiento de un modelo puede llevar mucho tiempo. Una vez un modelo está entrenado puede guardarse para moverlo a otro dispositivo, reutilizarlo, o continuar el entrenamiento más tarde.

Normalmente, se guarda un modelo entrenado en un archivo como un diccionario de Python, que contiene todos los parámetros y metadatos necesarios para reanudar el entrenamiento y hacer predicciones.

PyTorch tiene dos formas de guardar modelos: el método `save` y el método `load`. El método `save` guarda un modelo en un archivo, mientras que el método `load` carga un modelo de un archivo.

In [128]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Para cargar un modelo guardado, primero se crea una instancia de la clase del modelo y luego se llama al método `load_state_dict` para cargar los parámetros (pesos y sesgos de cada neurona) desde el fichero.

In [129]:
model = NeuralNetwork().to(device) # Creamos una instancia del modelo (con su arquitectura) y la movemos al dispositivo
model.load_state_dict(torch.load("model.pth")) # Cargamos los parámetros guardados del modelo

<All keys matched successfully>

## Usando el modelo para hacer predicciones

In [130]:
classes = [ # Clases de FashionMNIST
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval() # Establece el modelo en modo de evaluación
x, y = test_data[0][0], test_data[0][1] # Obtiene una imagen de prueba y su etiqueta
with torch.no_grad(): # Deshabilita el cálculo de gradientes
    x = x.to(device) # Movemos la imagen al dispositivo
    pred = model(x) # Obtenemos las predicciones
    predicted, actual = classes[pred[0].argmax(0)], classes[y] # Obtenemos la clase predicha y la clase real
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
